# User Defined Functions (UDFs)

Un UDF (User Defined Function) te permite crear una función personalizada en Python (o Scala, etc.) que se puede usar en expresiones SQL o DataFrame API.

##### Configuración del entorno

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark04").getOrCreate()
# Cargar el dataset de viajes
yellow_df = spark.read.parquet("../../data/yellow_tripdata_2023-01.parquet")
yellow_df.createOrReplaceTempView("yellow_trips")

#### Creación y aplicación de una UDF

In [ ]:
# Define una función en Python
def clasificar_viaje(distancia):
    if distancia < 2:
        return 'short'
    elif distancia <= 10:
        return 'medium'
    else:
        return 'long'

# Se registra como UDF
clasificar_udf = udf(clasificar_viaje, StringType())

# Se aplica a un DataFrame
df = yellow_df.withColumn("trip_type", clasificar_udf("trip_distance"))
df.select("trip_distance", "trip_type").show()

También puedes registrarla para usarla en SQL:

In [ ]:
spark.udf.register("clasificar_udf", clasificar_viaje, StringType())

spark.sql("""
    SELECT trip_distance, clasificar_udf(trip_distance) AS trip_type
    FROM yellow_trips
""").show()

## Persistencia de UDFs

Las UDFs en PySpark se definen y viven en la sesión actual del SparkSession o script donde se crean. No se persisten automáticamente en disco ni entre sesiones, como podría pasar con una función SQL en una base de datos relacional.

Sin embargo, cuando trabajas a nivel de producción con Spark en clústeres (como Databricks o EMR), puedes definir UDFs como funciones registradas en un paquete JAR, y cargarlas como funciones permanentes en Hive Metastore.

👉 Esto es más común con UDFs escritas en Scala (más eficientes que las de Python).

## Buenas prácticas con UDFs

En un enfoque estándar de ingeniería de datos, es muy común crear un archivo _.py_ (Por ejemplo: _utils.py_) para reutilizar el código de las funciones en distintas sesiones. Sin tener que escribirlo en cada notebook.

Simplemente tendríamos que llamar a la función que queremos importar desde cada notebook de la siguiente manera:

```python
from utils.py import clasificar_udf




Nuestro archivo _utils.py_ tendría un aspecto similar a este:
```python
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def clasificar_viaje(distancia):
    if distancia < 2:
        return 'short'
    elif distancia <= 10:
        return 'medium'
    else:
        return 'long'

clasificar_udf = udf(clasificar_viaje, StringType())

## Puntos clave de Spark SQL vs. API DataFrame

| Spark SQL                                    | DataFrame API                                    |
| -------------------------------------------- | ------------------------------------------------ |
| Muy familiar si vienes de SQL                | Más expresivo y flexible en lógica compleja      |
| Útil para prototipar y explorar              | Mejores herramientas de depuración en Python     |
| Puedes usar `.sql()` sobre vistas temporales | Puedes encadenar transformaciones más fácilmente |
| Igual de optimizado (Catalyst optimizer)     | Igual de rápido                                  |

Ambos se compilan al mismo plan lógico, así que rendimiento ≈ igual.
Es más una cuestión de preferencia y legibilidad.

##### Detenemos la sesión de Spark

In [ ]:
spark.stop()